# An SOSP model of encoding interference in verb choice and reading times
For a more complete description of the overall SOSP framework see the ClassicAgreementAttraction notebook. 

## A case of encoding interference
Here, we focus on a case of encoding interference in two sentence processing tasks. Encoding interference is when features of words in a sentence that are not relevant for retrieving those words for structure-building nonetheless interfere with parsing. The example we focus on here builds on the results of Barker, Nicol, & Garrett (2001). They used the sentence completion paradigm common in agreement attraction studies using subject NPs like the following:

1. The canoe by the cabin...
2. The canoe by the cabins...
3. The canoe by the sailboat...
4. The canoe by the sailboats...

Barker et al. manipulated the number marking on the second noun (N2; *cabin(s)* or *sailboat(s)*) and the semantic similarity between the first noun (N1; *canoe*) and the N2 (canoes and sailboats are both boats, while canoes and cabins have relatively little in common). They found significantly higher rates of agreement attraction in the semantically similar case than in the semantically dissimilar case. This suggests that the semantic similarity was interfering with participants' ability to choose the correct verb form, even though the semantic similarity is not, on most theories, a cue relevant for determining what the controller of verb number should be.

Another question is how long it takes to read or produce a verb after reading/producing the subject NP. We assume that doing so puts the parser in a particular state from which it must then settle in order to build the next bit of structure, i.e., incorporate a verb in the the existing parse. Which verb for the model settles on (the Barker et al. production results) and how long that takes (modeling new data we've gathered) are the main goals of this simulation.

## A simple SOSP approach to modeling the Barker et al. results
To model the Barker et al. results, we make the assumption that the subject NP has already been read/produced. We assume that this has pushed the verb's features toward expecting a the boat-like, singular N1 to be the subject. This is the initial condition for the model to start settling towards a parse. We define the dynamics by setting up a harmony landscape with peaks of different heights corresponding to different parses of varying degrees of wellformedness. We first set up the peaks, which are the centers of the RBFs that make up the global harmony function $HF$. The dimensions code the boat-features on the verb (0 = no boat, 1 = boat), the verb's number feature (0 = sg., 1 = pl.), and the strength of the N2-verb attachment link (we assume that the N1- and N2-verb links are in complementary distribution):

In [3]:
import numpy as np
# Here, we assume three identical boat features, which weights them quite heavily relative to the single number feature
dim_labels = ['boat1', 'boat2', 'boat3', 'pl', 'N2-attach']
condition_labels = ['canoe-kayaks', 'canoe-kayak', 'canoe-cabins', 'canoe-cabin']
ncond = len(condition_labels)

# Dimensions of the state space: [boat1, boat2, boat3, pl, ], +boat=1, +pl=1
# These are all of the attractors of the dynamics:
centers = np.array([[1.,1, 1, 0, 0],
                    [1, 1, 1, 0, 1],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 1],
                    [0, 0, 0, 1, 0],
                    [0, 0, 0, 1, 1],
                    [1, 1, 1, 1, 0],
                    [1, 1, 1, 1, 1]])
    
ndim = np.shape(centers)[1]

# Setting the initial conditions to reflect the bias N1 has already exerted on the verb, 
# starting it near the correct attractor
x0 = [0.6, 0.6, 0.6, 0.4, 0.4]

# We now calculate the heights of the harmony peaks
af = 0.8  # cost for failing to attach N1 and attaching N2 instead
mf = 0.2  # cost for having a mismatching feature

# Harmony values: 
# Columns are in same order as cond_labels
# Rows are in the same order as centers
harmony_vals = np.array([[       1,        1,        1,        1],
                         [   af*mf,       af, af*mf*mf,    af*mf],
                         [      mf,       mf,       mf,       mf],
                         [af*mf*mf,    af*mf,    af*mf,       af],
                         [   mf*mf,    mf*mf,    mf*mf,    mf*mf],
                         [   af*mf, af*mf*mf,       af,    af*mf],
                         [      mf,       mf,       mf,       mf],
                         [      af,    af*mf,    af*mf, af*mf*mf]])

The attachment failure and mismatching feature penalties effectively say that the system finds fragmentary speech tolerable and will parse it as such, but it prefers to avoid really bad grammatical errors (feature mismatches). Next, we prepare and run the simulations:

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

# Defining phi function
def phi(x, center, gamma):
    diff = x - center
    l2norm = np.sqrt(np.dot(diff, diff))
    phi = np.exp(-l2norm**2 / gamma)
    return phi


# A function for updating the state of the system according to the negative
# gradient of the harmony function
def step_dyn(x, centers, harmonies, gamma):
    dx = np.zeros(x.shape)
    for c in range(centers.shape[0]):
        dx += (-2./gamma * harmonies[c]
               * (x - centers[c,:]) * phi(x, centers[c,:], gamma))
    return dx


# Proximity to a fixed point
def not_close(x, centers, tol):
    for c in range(centers.shape[0]):
        diff = x - centers[c]
        l2norm = np.sqrt(np.dot(diff, diff))
        if l2norm < tol:
            #print(centers[c])
            return False
        else:
            return True

# Find out which fp. the system reached
def which_attr(x):
    x = np.round(x)   # Note:  assumes all centers are at corners of [0, 1]^ndim
    for c in range(centers.shape[0]):
        if np.all(x == centers[c,]):
            return c
    return -1


# Singular or plural verb?
def sg_pl(x):
    x = np.round(x)
    if x[-2] == 0:
        return 0
    elif x[-2] == 1:
        return 1
    else:
        return -1


# Setting parameters
gamma = 0.5
maxsteps = 1000
tau = 0.01
tol = 0.2
noisemag = 0.05
nruns = 1000

# Running
rts = np.zeros((nruns, ncond))
parses = np.zeros((nruns, ncond))
for cond in range(ncond):
    print('\nCondition: {}'.format(condition_labels[cond]))
    print('Of {}: '.format(nruns), end='')
    for run in range(nruns):
        if (run+1) % 100 == 0:
            print('[{}] '.format(run+1), end='')
        x = np.zeros((maxsteps, centers.shape[1]))
        x[0,] = x0
        if nruns is not 1:
            noise = np.random.normal(0, 1, x.shape)
        else:
            noise = np.zeros(x.shape)

        t = 0
        while t < maxsteps-1:
            if not_close(x[t,], centers, tol = tol):
                x[t+1,] = x[t,]+(tau*(step_dyn(x[t,], centers, harmony_vals[:,cond], gamma))
                                 + np.sqrt(noisemag*tau)*noise[t,])
                t += 1
            else:
                break

        xtrunc = x[~np.all(x == 0, axis=1)]
        rts[run, cond] = xtrunc.shape[0]  # Time is noun attraction time, not verb time (???)
        #parses[run, cond] = which_attr(xtrunc[-1,])
        parses[run, cond] = sg_pl(xtrunc[-1,])
    if nruns == 1:
        plt.plot(xtrunc[:,ndim-1], xtrunc[:,0], label=condition_labels[cond])  # Boat-Attach
        ftitle = 'Boat vs. Attachment'# Boat-Number
    
if nruns == 1:
    plt.ylim(-0.1, 1.1)
    plt.xlim(-0.1, 1.1)
    plt.legend()
    plt.title(ftitle)
    plt.show()
    print(rts[0,])
else:
    print('\nParses:\n')
    parse_labels = ['Vsg', 'Vpl', 'other']
    for cond in range(ncond):
        uniq, cts = np.unique(parses[:,cond], return_counts=True)
        print('{}:'.format(condition_labels[cond]))
        for u in range(len(uniq)):
            if cts[u] is not 0:
                print('\t{}:\t{}'.format(parse_labels[u], cts[u]/nruns))
    print('\nMean =\t{}\nSD =\t{}'.format(np.mean(rts, axis=0), np.round(np.std(rts, axis=0), 3)))
    for cond in range(4):
        # Density plot of settling times
        sns.kdeplot(rts[:,cond], label=condition_labels[cond])
    plt.title('Density plot of times to settle to an attractor')
    plt.ylim(0, 0.025)
    plt.show()


Condition: canoe-kayaks
Of 1000: [100] [200] [300] [400] [500] [600] [700] [800] [900] [1000] 
Condition: canoe-kayak
Of 1000: [100] [200] [300] [400] [500] [600] [700] [800] [900] [1000] 
Condition: canoe-cabins
Of 1000: [100] [200] [300] [400] [500] [600] [700] [800] [900] [1000] 
Condition: canoe-cabin
Of 1000: [100] [200] [300] [400] [500] [600] [700] [800] [900] [1000] 
Parses:

canoe-kayaks:
	Vsg:	0.953
	Vpl:	0.047
canoe-kayak:
	Vsg:	1.0
canoe-cabins:
	Vsg:	0.995
	Vpl:	0.005
canoe-cabin:
	Vsg:	1.0

Mean =	[ 161.485  185.079  104.474  106.183]
SD =	[ 195.018  129.018   73.008   87.063]


NameError: name 'cond_labels' is not defined

## Discussion
Overall, we this simulation replicates Barker et al. (2001)'s finding of increased agreement attraction when N1 and N2 are semantically similar compared to when they are semantically dissimilar. Moreover, it makes the novel prediction that reading times at the verb should be slower in the similar condition than in the dissimilar condition. This prediction has received initial support from new self-paced reading data in Smith, Franck, and Tabor (in progress), where we found a main effect of semantic similarity at the verb in the same direction as the simulations here.

What’s happening in this simulation is that the model is mostly building a parse where it attaches the verb to N1, but sometimes attaching to N2 and it's generating a verb that matches the features of N2 on the Boat and Number dimensions.  It does this relatively often for *canoe by the kayaks* because of it's initial journey in the direction of forming the parse for *canoe is* starts turning on all the +Boat features and thus brings it somewhat near the state that it need to be in to take *kayaks* as a subject (the noise therefore sometimes manages to bump into this state.). It does this relatively rarely for *canoe by the cabins* because *cabins* needs all the boat features turned off so the state of having *cabins* as subject is further away (it is not as likely that the noise to jostle it into this state).

Initially, the state starts off completely unbiased in all features, then canoe is read, and this pushes the features a little bit toward +Boat, -Plural. Then N2 is read, and the potential of attaching the verb to N2 pulls the features in various directions. When N2 is *kayak*, the system is pulled toward attachment to N2 with treelet features at [+Boat (x3), -Plural]. This state is quite near attachment to N1 (the correct attachment)---the only thing that’s different about it is the link value and the missing attachments to N1 so it exerts very strong competition. When N2 is *kayaks*, the pull is still strong, but a little less so because the plural feature of *kayaks* makes N2 attachment less similar to N1 attachment. But if N2 is either *cabin* or *cabins*, the pull is relatively weak since all the semantic feature differences make these parses quite different from the current state. After some settling under the influence of N2, the verb itself is read. In order to finish processing the verb, the system must get reasonably close (tol = 0.2, Euclidean distance) to a stable state. The verb, which is *is*, exerts a force that pushes the system toward the singular state. In the condition *canoe near kayak*, there is high resistance to the force of *is* from the possibility of making *kayak* the subject. In the condition *canoe near kayaks*, there is still quite a bit of resistance, though it's a little less, as described above. In the *cabin* and *cabins* condition, there is relatively little resistance.   In other words, *kayak* is slowest, *kayaks* is faster, but still pretty slow; *cabin* and *cabins* are both faster.    (Note that the time to process *is* is closely correlated with the time it would have taken the system to reach the parse of *is* based on just perceiving *(the) N1 (near the) N2*. Because it was simpler to program, the times reported above are actually simply the times to convergence after processing N2).

It is true that the model actually generates both a main effect of Similarity and possibly an interaction between N2 number and semantic similarity, whereas we only observed a main effect of similarity. We assume this is because the experimental design was not powerful enough to detect the interaction.